# Provincial Fertilization Program - Summary Attributes

In [ ]:
# Import Python modules
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import gc as garc
import scipy.stats as stats
from fcgadgets.macgyver import utilities_general as gu
from fcgadgets.macgyver import utilities_gis as gis
from fcgadgets.macgyver import utilities_inventory as invu
from fcgadgets.cbrunner import cbrun_utilities as cbu
from fcgadgets.cbrunner import cbrun_utilities as cbu


In [ ]:
# Set figure properties
fs=6
params={'font.sans-serif':'Arial','font.size':fs,'axes.edgecolor':'black','axes.labelsize':fs,'axes.labelcolor':'black','axes.titlesize':fs,'axes.linewidth':0.5,'lines.linewidth':0.5,
        'text.color':'black','xtick.color':'black','xtick.labelsize':fs,'xtick.major.width':0.5,'xtick.major.size':3,'xtick.direction':'in','ytick.color':'black','ytick.labelsize':fs,
        'ytick.major.width':0.5,'ytick.major.size':3,'ytick.direction':'in','legend.fontsize':fs,'savefig.dpi':300,'savefig.transparent':True}
plt.rcParams.update(params)

In [ ]:
# Initiate metadata structure and import paths
meta={}
meta=gu.ipickle(r'D:\Data\FCI_Projects\SummaryNutrientManagementFull\Inputs\Metadata.pkl')
meta['Paths']['Geospatial']=meta['Paths']['Project'] + '\\Geospatial'
meta['Paths']['Model Code']=r'C:\Users\rhember\Documents\Code_Python\fcgadgets\cbrunner'
meta['Paths']['Taz Datasets']=r'C:\Users\rhember\Documents\Data\Taz Datasets'

In [ ]:
#Configuration
meta['Project']={}
meta['Project']['N Stand']=sxy['x'].size
meta=cbu.ImportProjectConfig(meta)

In [ ]:
# Open invnetory files
sxy,atu,op,burnsev,vri,fcinv,fcsilv,fcres,pl,fire,pest,cut,lul,park,ogmal=invu.LoadSparseGeospatialInputs(meta)
idx=invu.LoadSparseGridIndex(meta)

In [ ]:
# Extract disturbance/management event chronology from inventory layers
flag_use_saved_file=1
if flag_use_saved_file==0:
    dmec=invu.PrepDMEC(idx,meta,atu,pl,op,fcinv,vri,cut,fire,burnsev,pest)
    gu.opickle(meta['Paths']['Project'] + '//Inputs//dmec.pkl',dmec)
else:
    dmec=gu.ipickle(meta['Paths']['Project'] + '//Inputs//dmec.pkl')

In [ ]:
# Define strings that frequently need to be populated with zeros
StringsToFill=['Month','Day','SILV_FUND_SOURCE_CODE','FIA_PROJECT_ID','OPENING_ID','ACTUAL_TREATMENT_AREA','ACTUAL_PLANTED_NUMBER', \
               'PL_SPECIES_CD1','PL_SPECIES_PCT1','PL_SPECIES_GW1','PL_SPECIES_CD2','PL_SPECIES_PCT2','PL_SPECIES_GW2', \
               'PL_SPECIES_CD3','PL_SPECIES_PCT3','PL_SPECIES_GW3','PL_SPECIES_CD4','PL_SPECIES_PCT4','PL_SPECIES_GW4', \
               'PL_SPECIES_CD5','PL_SPECIES_PCT5','PL_SPECIES_GW5']

In [ ]:
# Exclude duplicate events
if meta['Project']['Exclude duplicate events']=='On':
    dmec=invu.Exclude_Duplicate_Events(meta,dmec)

In [ ]:
# Exclude unidentified activities or disturbances
if meta['Project']['Exclude unidentified events']=='On':
    dmec=invu.Exclude_Unidentified_Events(meta,dmec)

In [ ]:
# Put events in order
dmec=invu.PutEventsInOrder(dmec,meta)